In [1]:
from Q_learning import *
import pickle


Word_set = cache(total_words=8885) #max count

class Hangman2:
    def __init__(self, word):
        self.word = word
        self.guessed_word = ["_"] * len(self.word)
        self.lives = 5
        self.guessed_letters = []

    def __str__(self):
        return " ".join(self.guessed_word)

    def guess(self, letter):
        if len(letter) != 1 or not letter.isalpha():
            return "Invalid input"

        letter = letter.lower()
        if letter in self.guessed_letters:
            return "Already guessed"

        self.guessed_letters.append(letter)
        if letter in self.word:
            for i, char in enumerate(self.word):
                if char == letter:
                    self.guessed_word[i] = letter
            return "Correct"
        else:
            self.lives -= 1
            return "Incorrect"

    def is_game_over(self):
        if "_" not in self.guessed_word:
            return "win"
        elif self.lives <= 0:
            return "lose"
        return None

    def get_state(self):
        return (self.guessed_word.copy(), self.guessed_letters.copy())


def evaluate_agent2(agent, games=10, word_set=Word_set):
    if word_set is None or len(word_set) < games:
        raise ValueError("Word set must contain at least as many words as games to play.")

    wins = 0
    total_lives_left = 0

    # Disable exploration for evaluation
    original_exploration = agent.exploration_rate
    agent.exploration_rate = 0

    # Verify Q-table is loaded
    if not agent.q_table:
        print("⚠️ WARNING: Q-table is empty!")
        return

    for game in range(games):
        word = word_set[game]
        hangman = Hangman2(word)

        while True:
            state = hangman.get_state()
            normalized_state = agent.normalize_state(state)

            action = agent.choose_action(normalized_state, hangman.guessed_letters)

            if action is None:
                break  # No valid actions left

            _ = hangman.guess(action)  
            
            game_status = hangman.is_game_over()
            if game_status:
                if "win" in game_status:
                    wins += 1
                    total_lives_left += hangman.lives
                break

    # Restore original exploration rate
    agent.exploration_rate = original_exploration

    # Summary
    win_rate = wins / games
    avg_lives = total_lives_left / wins if wins > 0 else 0

    print("\n🎯 Evaluation Results:")
    print(f"Games played: {games}")
    print(f"Games won: {wins}")
    print(f"Win rate: {win_rate * 100:.2f}%")
    print(f"Average lives left (wins only): {avg_lives:.2f}")



In [2]:
agent = Agent()


with open("q_table_huge.pkl", "rb") as f:
    agent.q_table = pickle.load(f)

evaluate_agent2(agent, games=100)


🎯 Evaluation Results:
Games played: 100
Games won: 7
Win rate: 7.00%
Average lives left (wins only): 2.57


A 600 mb file of dictionary of Q-values provide not so compelling results.

Lets instead try a more traditional way of solving this

In [3]:
def Allcache(total_words=500_000_000):
    url  = f"https://random-word-api.herokuapp.com/word?number={total_words}&length=5"

    response = requests.get(url)
    if response.status_code == 200:
        word_list = response.json() 
    return (word_list)

word_list = Allcache()
len(word_list)

8885

In [20]:
def remove_words(incorrect_letters, word_list):
    if not incorrect_letters:
        return word_list
    
    incorrect_set  = set(incorrect_letters)
    newlist = []
    
    for word in word_list:
        found_wrong_letter = False
        for letter in incorrect_set:
            if letter in word:
                found_wrong_letter = True
                break
        if not found_wrong_letter:
            newlist.append(word)
    
    return newlist

def find_letters(gussed_word, gussed_letters):
    """
    Prints:
        Incorrct_letters
        Correct_letters
    """
    correct_set = set(find_correct_letters(gussed_word))
    
    incorrect = []
    for letter in gussed_letters:
        if letter not in correct_set:
            incorrect.append(letter)

    return incorrect, list(correct_set)

def find_correct_letters(gussed_word):    
    correct_set = set()

    for item in gussed_word:
        if item != "_" and item != " ":
            correct_set.add(item)

    return list(correct_set)

def keep_words(guessed_word, word_list):
    if not guessed_word:
        return word_list

    newlist = []
    for word in word_list:
        if len(word) != len(guessed_word):
            continue  # skip words with different length

        match = True
        for i in range(len(guessed_word)):
            if guessed_word[i] != "_" and guessed_word[i] != word[i]:
                match = False
                break

        if match:
            newlist.append(word)
    return newlist




def most_common_letter(word_list, guessed_words):
    dictionary = {}
    for word in word_list:
        for letter in word:
            if letter in dictionary:
                dictionary[letter] += 1
            else:
                dictionary[letter] = 1

    for item in guessed_words:
        dictionary[item] = 0

    return max(dictionary, key=dictionary.get)

In [21]:
def evaluate_agent_simple(games=10, word_set=Word_set):
    if word_set is None or len(word_set) < games:
        raise ValueError("Word set must contain at least as many words as games to play.")

    wins = 0
    total_lives_left = 0


    for game in range(games):
        word = word_set[game]
        hangman = Hangman2(word)
        curr_word_set = word_set
        incorrect_letters = None
        while True:
            guessed_letters = hangman.guessed_letters
            guessed_word = hangman.guessed_word
    
            # Convert guessed_letters to a set once for fast lookup
            guessed_letters_set = set(guessed_letters)

            # Choose next best letter to guess
            action = most_common_letter(curr_word_set, guessed_letters_set)
            
            move = hangman.guess(action)

            game_status = hangman.is_game_over()
            if game_status:
                if "win" in game_status:
                    wins += 1
                    total_lives_left += hangman.lives
                break


            if move == "incorrect":
                incorrect_letters = action
            else:
                # Apply filters to reduce the word set efficiently
                curr_word_set = keep_words(guessed_word, curr_word_set)

            if incorrect_letters:
                curr_word_set = remove_words(incorrect_letters, curr_word_set)

            if not action or not curr_word_set:
                print(f"action: {action}, Curr_word_set: {curr_word_set}")
                break



    # Summary
    win_rate = wins / games
    avg_lives = total_lives_left / wins if wins > 0 else 0

    print("\n🎯 Evaluation Results:")
    print(f"Games played: {games}")
    print(f"Games won: {wins}")
    print(f"Win rate: {win_rate * 100:.2f}%")
    print(f"Average lives left (wins only): {avg_lives:.2f}")



In [22]:
evaluate_agent_simple(games=8885)


🎯 Evaluation Results:
Games played: 8885
Games won: 3524
Win rate: 39.66%
Average lives left (wins only): 2.19


In [7]:
evaluate_agent2(agent, games=8885)


🎯 Evaluation Results:
Games played: 8885
Games won: 901
Win rate: 10.14%
Average lives left (wins only): 3.06
